In [4]:
%pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 903.7 kB/s eta 0:00:0000:01:010m
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Reshape, Conv2DTranspose, Conv2D, MaxPooling2D, BatchNormalization, Activation, Add, Multiply, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from skimage import exposure, filters
import cv2

# Load and preprocess real retinal images
train_dir = "/Users/hiteshwaralavala/Desktop/data_set_1/train"
test_dir = "/Users/hiteshwaralavala/Desktop/data_set_1/test"
image_size = (64, 64)
batch_size = 8

train_csv = "/Users/hiteshwaralavala/Desktop/igb_dataset_1/train.csv"
test_csv = "/Users/hiteshwaralavala/Desktop/igb_dataset_1/test.csv"

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

# Define function to preprocess image
def preprocess_image(img):
    # Gaussian blur
    blurred_img = cv2.GaussianBlur(img, (5, 5), 0)
    # Histogram equalization
    equalized_img = exposure.equalize_hist(blurred_img)
    return equalized_img

# Create ImageDataGenerator with preprocessing function
train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_image
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_image
)

# Define train and test generators using flow_from_dataframe
# Define train and test generators using flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col="Image",  # Column containing image filenames
    y_col="Label",  # Column containing labels
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Changed class_mode to "categorical"
    shuffle=True
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_dir,
    x_col="Image",  # Column containing image filenames
    y_col="Label",  # Column containing labels
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Changed class_mode to "categorical"
    shuffle=False  # No need to shuffle test data
)

# Define Squeeze-and-Excitation (SE) block
def se_block(input_tensor, ratio=16):
    num_channels = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(num_channels // ratio, activation='relu')(se)
    se = Dense(num_channels, activation='sigmoid')(se)
    return Multiply()([input_tensor, se])

# Define Discriminator architecture
def build_discriminator(input_shape=(64, 64, 3)):
    model = Sequential([
        Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(128, (3, 3), strides=(2, 2), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        GlobalAveragePooling2D(),
        Dense(1, activation='sigmoid')
    ])
    return model

# Define Generator architecture
def build_generator(latent_dim):
    model = Sequential([
        Dense(8 * 8 * 256, input_dim=latent_dim),
        Reshape((8, 8, 256)),
        Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', activation='sigmoid')
    ])
    return model

# Instantiate Discriminator and Generator
latent_dim = 100
discriminator = build_discriminator()
generator = build_generator(latent_dim)

# Define GAN model
discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')
discriminator.trainable = False

gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

# Train the GAN
epochs = 10
for epoch in range(epochs):
    for batch in train_generator:
        real_images = batch  # Only unpack the input images, ignore the labels
        # Process real_images and train your models
        real_images = real_images.reshape(batch_size, 64, 64, 3)
        # Train Discriminator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_images = generator.predict(noise)
        discriminator_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
        discriminator_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
        discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generator_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        print(f"Epoch {epoch+1}/{epochs}, Batch {batch}/{len(train_generator)}, "
              f"Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}")

# Fine-tune the InceptionV3 model with SE block
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

x = base_model.output
x = se_block(x)  # Add SE block
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Save the final model
model.save("final-1_diabetic_retinopathy_detection_model.h5")

test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)


Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 28099 invalid image filename(s) in x_col="Image". These filename(s) will be ignored.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 7027 invalid image filename(s) in x_col="Image". These filename(s) will be ignored.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument

AttributeError: 'tuple' object has no attribute 'reshape'

In [10]:
print("Shape of real_images:", real_images.shape)


Shape of real_images: (8, 64, 64, 3)
